In [ ]:
import os
import requests
from dotenv import load_dotenv

# --- last .env.local hvis den finnes ---
load_dotenv(".env.local")

token = os.getenv("AIRTABLE_TOKEN")
base_id = os.getenv("AIRTABLE_BASE_ID")
table_id = os.getenv("AIRTABLE_TABLE_ID")

url = f"https://api.airtable.com/v0/{base_id}/{table_id}?maxRecords=1"

headers = {"Authorization": f"Bearer {token}"}
response = requests.get(url, headers=headers)
data = response.json()

print("Status:", response.status_code)
if "records" in data and data["records"]:
    record = data["records"][0]
    print("\nFeltnøkler i første record:")
    print(list(record["fields"].keys()))
    print("\nEksempeldata:")
    for k, v in record["fields"].items():
        print(f"{k}: {v}")
else:
    print("Ingen records funnet eller feil i API-kallet.")


In [1]:
print(123)

123


In [4]:
# print_interest_short.py
import os
import sys
import json
import subprocess
import re
from pathlib import Path

if hasattr(sys.stdout, "reconfigure"):
    sys.stdout.reconfigure(encoding="utf-8")

# Oppsett
ALLOW_DIRS = ["src/lib", "src/app", "src/components"]
ALLOW_EXTS = {".ts", ".tsx", ".js", ".mjs", ".cjs", ".json", ".css"}
EXCLUDE_SUBSTR = {"__tests__", ".test.", ".spec."}

INCLUDE_CONFIG = True
CONFIG_FILES = {
    "package.json",
    "next.config.js",
    "tailwind.config.ts",
    "postcss.config.mjs",
    "tsconfig.json",
    "next-env.d.ts",
    "src/app/globals.css",
    "vercel.json",
}

SKIP_DIRS = {"node_modules", ".next", ".git", ".vercel", "__pycache__", ".venv", ".idea", ".vscode"}

LANG_BY_EXT = {
    ".ts": "ts",
    ".tsx": "tsx",
    ".js": "javascript",
    ".mjs": "javascript",
    ".cjs": "javascript",
    ".json": "json",
    ".css": "css",
}

SAMPLE_ENV_KEYS = int(os.getenv("PI_SAMPLE_ENV_KEYS", "5"))
MAX_FILES_IN_SUMMARY = int(os.getenv("PI_MAX_FILES_IN_SUMMARY", "9999"))

# -------- helpers --------
def is_interesting(rel_path: Path) -> bool:
    p = rel_path.as_posix()
    if any(x in p for x in EXCLUDE_SUBSTR):
        return False
    if any(p.startswith(d) for d in ALLOW_DIRS) and rel_path.suffix in ALLOW_EXTS:
        return True
    if INCLUDE_CONFIG and p in CONFIG_FILES:
        return True
    return False

def iter_files(root: Path):
    for dirpath, dirnames, filenames in os.walk(root):
        dirnames[:] = [d for d in dirnames if d not in SKIP_DIRS]
        for fn in filenames:
            p = Path(dirpath) / fn
            rel = p.relative_to(root)
            if is_interesting(rel):
                yield rel

def run_git(cmd):
    return subprocess.check_output(cmd, stderr=subprocess.DEVNULL).decode().strip()

def try_git_info():
    info = {}
    try:
        info["commit"] = run_git(["git", "rev-parse", "HEAD"])
        info["branch"] = run_git(["git", "rev-parse", "--abbrev-ref", "HEAD"])
        status = ""
        try:
            status = run_git(["git", "status", "--porcelain"])
        except Exception:
            pass
        info["state"] = "dirty" if status else "clean"
    except Exception:
        info = {"commit": None, "branch": None, "state": None}
    return info

def read_pkg_versions():
    out = {"next": None, "react": None, "react-dom": None, "node": None, "typescript": None}
    try:
        pkg = json.loads(Path("package.json").read_text(encoding="utf-8"))
        deps = {**pkg.get("dependencies", {}), **pkg.get("devDependencies", {})}
        out["next"] = deps.get("next")
        out["react"] = deps.get("react")
        out["react-dom"] = deps.get("react-dom")
        out["typescript"] = deps.get("typescript")
        out["node"] = pkg.get("engines", {}).get("node")
    except Exception:
        pass
    return out

def read_env_keys():
    result = []
    for envfile in [".env", ".env.local", ".env.development", ".env.production"]:
        p = Path(envfile)
        if not p.exists():
            continue
        try:
            keys = []
            for line in p.read_text(encoding="utf-8", errors="ignore").splitlines():
                line = line.strip()
                if not line or line.startswith("#") or "=" not in line:
                    continue
                k, _ = line.split("=", 1)
                keys.append(k.strip())
            if keys:
                uniq = sorted(set(keys))
                result.append({
                    "file": envfile,
                    "keys_count": len(uniq),
                    "sample": uniq[:SAMPLE_ENV_KEYS]
                })
        except Exception:
            continue
    return result

NEXT_SPECIAL = {"page.tsx": "page", "layout.tsx": "layout", "route.ts": "route", "loading.tsx": "loading", "error.tsx": "error"}

EXPORT_PATTERNS = [
    re.compile(r'^\s*export\s+default\s+function\s+([A-Za-z0-9_]+)', re.MULTILINE),
    re.compile(r'^\s*export\s+function\s+([A-Za-z0-9_]+)', re.MULTILINE),
    re.compile(r'^\s*export\s+class\s+([A-Za-z0-9_]+)', re.MULTILINE),
    re.compile(r'^\s*export\s+(?:const|let|var)\s+([A-Za-z0-9_]+)', re.MULTILINE),
    re.compile(r'^\s*export\s+type\s+([A-Za-z0-9_]+)', re.MULTILINE),
    re.compile(r'^\s*export\s+interface\s+([A-Za-z0-9_]+)', re.MULTILINE),
]

REACT_COMPONENT_PATTERNS = [
    re.compile(r'^\s*function\s+([A-Z][A-Za-z0-9_]*)\s*\(', re.MULTILINE),
    re.compile(r'^\s*(?:const|let|var)\s+([A-Z][A-Za-z0-9_]*)\s*[:=]\s*\(?\s*\(', re.MULTILINE),
]

def detect_next_role(path: Path) -> str | None:
    name = path.name
    if name in NEXT_SPECIAL:
        return NEXT_SPECIAL[name]
    if name in {"page.js", "layout.js", "route.js", "loading.js", "error.js"}:
        return name.split(".")[0]
    return None

def extract_outline(path: Path, root: Path):
    abs_p = root / path
    ext = path.suffix
    role = detect_next_role(path)
    outline = {"path": path.as_posix(), "bytes": 0, "lines": 0, "lang": LANG_BY_EXT.get(ext, ""), "next_role": role, "exports": [], "components": [], "json_keys": [], "css_rules": 0}
    try:
        text = abs_p.read_text(encoding="utf-8", errors="replace")
        outline["bytes"] = abs_p.stat().st_size
        outline["lines"] = text.count("\n") + 1
    except Exception:
        return outline

    if ext in {".ts", ".tsx", ".js", ".mjs", ".cjs"}:
        exports = set()
        for pat in EXPORT_PATTERNS:
            exports.update(pat.findall(text))
        comps = set()
        for pat in REACT_COMPONENT_PATTERNS:
            comps.update(pat.findall(text))
        outline["exports"] = sorted(exports)[:50]
        outline["components"] = sorted(comps)[:50]

    elif ext == ".json":
        try:
            data = json.loads(text)
            if isinstance(data, dict):
                outline["json_keys"] = list(data.keys())[:50]
        except Exception:
            pass

    elif ext == ".css":
        # grovt estimat på regler
        outline["css_rules"] = len([m for m in re.finditer(r'{', text)])
    return outline

def build_src_tree(root: Path, files):
    # Kompakt tre for src/
    src = root / "src"
    lines = []
    if not src.exists():
        return ["(ingen src/)"]
    files_set = {f.as_posix() for f in files}
    for dirpath, dirnames, filenames in os.walk(src):
        dirnames[:] = [d for d in dirnames if d not in {"__tests__"}]
        rel = Path(dirpath).relative_to(root).as_posix()
        depth = rel.count("/")
        indent = "  " * depth
        lines.append(f"{indent}{Path(dirpath).name}/")
        for fn in sorted(filenames):
            relpath = Path(dirpath).relative_to(root) / fn
            rp = relpath.as_posix()
            if any(s in fn for s in ["test.", "spec."]):
                continue
            if rp in files_set:
                lines.append(f"{indent}  {fn}")
    return lines

# -------- main --------
def main():
    root = Path(".").resolve()
    files = sorted(set(iter_files(root)), key=lambda p: p.as_posix())
    git = try_git_info()
    versions = read_pkg_versions()
    env_meta = read_env_keys()

    outlines = []
    total_bytes = 0
    for i, p in enumerate(files):
        o = extract_outline(p, root)
        outlines.append(o)
        total_bytes += o.get("bytes", 0)
        if i + 1 >= MAX_FILES_IN_SUMMARY:
            break

    summary = {
        "project_root": str(root),
        "counts": {"files_listed": len(outlines), "sum_bytes_listed": total_bytes},
        "git": git,
        "versions": versions,
        "env_files": env_meta,
    }

    # __SUMMARY__
    print("```json title=__SUMMARY__.json")
    print(json.dumps(summary, ensure_ascii=False, indent=2))
    print("```")
    print()

    # __SRC_TREE__
    print("```text title=__SRC_TREE__.txt")
    for line in build_src_tree(root, files):
        print(line)
    print("```")
    print()

    # __OUTLINE__
    print("```json title=__OUTLINE__.json")
    print(json.dumps(outlines, ensure_ascii=False, indent=2))
    print("```")

if __name__ == "__main__":
    main()


```json title=__SUMMARY__.json
{
  "project_root": "C:\\Users\\TorkelWestby\\inspiration_bank",
  "counts": {
    "files_listed": 30,
    "sum_bytes_listed": 210451
  },
  "git": {
    "commit": "46bbb88d5103c7b5f758a97e95d576fdc3170b23",
    "branch": "main",
    "state": "dirty"
  },
  "versions": {
    "next": "14.2.15",
    "react": "^18.3.1",
    "react-dom": "^18.3.1",
    "node": null,
    "typescript": "^5"
  },
  "env_files": [
    {
      "file": ".env.local",
      "keys_count": 15,
      "sample": [
        "AIRTABLE_BASE_ID",
        "AIRTABLE_TABLE_ID",
        "AIRTABLE_TOKEN",
        "AI_ANALYZE_ENABLED",
        "AI_MAX_REQ_PER_HOUR"
      ]
    }
  ]
}
```

```text title=__SRC_TREE__.txt
src/
  app/
    globals.css
    layout.tsx
    page.tsx
    api/
      ai-analyze/
        route.ts
      auth/
        login/
          route.ts
      ideas/
        route.ts
    ideas/
      page.tsx
    kanban/
      page.tsx
  components/
    DeleteConfirmModal.tsx
    FormTypeSe

In [2]:
# print_interest.py
import os
import sys
import json
import subprocess
from pathlib import Path

# Sikrer riktig tegnsett
if hasattr(sys.stdout, "reconfigure"):
    sys.stdout.reconfigure(encoding="utf-8")

# Oppsett
ALLOW_DIRS = ["src/lib", "src/app", "src/components"]
ALLOW_EXTS = {".ts", ".tsx", ".js", ".mjs", ".cjs", ".json", ".css"}
EXCLUDE_SUBSTR = {"__tests__", ".test.", ".spec."}

INCLUDE_CONFIG = True
CONFIG_FILES = {
    "package.json",
    "next.config.js",
    "tailwind.config.ts",
    "postcss.config.mjs",
    "tsconfig.json",
    "next-env.d.ts",
    "src/app/globals.css",
    "vercel.json",
}

# Grenser kan overstyres via env
MAX_BYTES = int(os.getenv("PI_MAX_BYTES", "200000"))
MAX_LINES = int(os.getenv("PI_MAX_LINES", "400"))

# Filer og mapper å hoppe over
SKIP_DIRS = {"node_modules", ".next", ".git", ".vercel", "__pycache__", ".venv", ".idea", ".vscode"}

LANG_BY_EXT = {
    ".ts": "ts",
    ".tsx": "tsx",
    ".js": "javascript",
    ".mjs": "javascript",
    ".cjs": "javascript",
    ".json": "json",
    ".css": "css",
}

def is_interesting(path: Path) -> bool:
    p = path.as_posix()
    if any(x in p for x in EXCLUDE_SUBSTR):
        return False
    if any(p.startswith(d) for d in ALLOW_DIRS) and path.suffix in ALLOW_EXTS:
        return True
    if INCLUDE_CONFIG and p in CONFIG_FILES:
        return True
    return False

def iter_files(root: Path):
    for dirpath, dirnames, filenames in os.walk(root):
        dirnames[:] = [d for d in dirnames if d not in SKIP_DIRS]
        for fn in filenames:
            p = Path(dirpath) / fn
            # Normaliser til prosjektrot
            rel = p.relative_to(root).as_posix()
            if is_interesting(Path(rel)):
                yield Path(rel)

def try_git_info():
    def run(cmd):
        return subprocess.check_output(cmd, stderr=subprocess.DEVNULL).decode().strip()

    info = {}
    try:
        info["commit"] = run(["git", "rev-parse", "HEAD"])
        info["branch"] = run(["git", "rev-parse", "--abbrev-ref", "HEAD"])
        # Dirty om det finnes endringer
        dirty = ""
        try:
            status = run(["git", "status", "--porcelain"])
            dirty = "dirty" if status else "clean"
        except Exception:
            pass
        info["state"] = dirty or "unknown"
    except Exception:
        info["commit"] = None
        info["branch"] = None
        info["state"] = None
    return info

def read_pkg_versions():
    versions = {"next": None, "react": None, "react-dom": None, "node": None}
    try:
        pkg = json.loads(Path("package.json").read_text(encoding="utf-8"))
        deps = {**pkg.get("dependencies", {}), **pkg.get("devDependencies", {})}
        versions["next"] = deps.get("next")
        versions["react"] = deps.get("react")
        versions["react-dom"] = deps.get("react-dom")
        engines = pkg.get("engines", {})
        versions["node"] = engines.get("node")
    except Exception:
        pass
    return versions

def read_env_keys():
    keys = {}
    for envfile in [".env", ".env.local", ".env.development", ".env.production"]:
        p = Path(envfile)
        if not p.exists():
            continue
        try:
            file_keys = []
            for line in p.read_text(encoding="utf-8", errors="ignore").splitlines():
                line = line.strip()
                if not line or line.startswith("#") or "=" not in line:
                    continue
                k, _ = line.split("=", 1)
                file_keys.append(k.strip())
            if file_keys:
                keys[envfile] = sorted(set(file_keys))
        except Exception:
            continue
    return keys

def lang_for(path: Path) -> str:
    return LANG_BY_EXT.get(path.suffix, "")

def print_header(summary):
    # Kort JSON som Claude kan lese maskinelt
    print("```json title=__SUMMARY__.json")
    print(json.dumps(summary, ensure_ascii=False, indent=2))
    print("```")
    print()

def print_src_tree(root: Path, files):
    # Kompakt tre kun for src
    print("```text title=__SRC_TREE__.txt")
    src = root / "src"
    if not src.exists():
        print("(ingen src/)")
        print("```")
        print()
        return

    sizes = {f.as_posix(): (root / f).stat().st_size if (root / f).exists() else 0 for f in files}

    for dirpath, dirnames, filenames in os.walk(src):
        dirnames[:] = [d for d in dirnames if d not in {"__tests__"}]
        rel = Path(dirpath).relative_to(root).as_posix()
        depth = rel.count("/")
        indent = "  " * depth
        print(f"{indent}{Path(dirpath).name}/")
        for fn in sorted(filenames):
            relpath = Path(dirpath).relative_to(root) / fn
            rp = relpath.as_posix()
            if any(s in fn for s in ["test.", "spec."]):
                continue
            if Path(fn).suffix in ALLOW_EXTS or rp.endswith("globals.css"):
                sz = sizes.get(rp, 0)
                print(f"{indent}  {fn}  ({sz} bytes)")
    print("```")
    print()

def print_file(root: Path, p: Path):
    abs_p = root / p
    size = abs_p.stat().st_size if abs_p.exists() else 0
    truncated = False
    print(f"```{lang_for(p)} title={p.as_posix()}")
    try:
        if size > MAX_BYTES:
            print(f"// HOPPER OVER: filstørrelse {size} > {MAX_BYTES} bytes")
            print("```")
            print()
            return
        text = abs_p.read_text(encoding="utf-8", errors="replace")
        lines = text.splitlines()
        if len(lines) > MAX_LINES:
            half = MAX_LINES // 2
            head = "\n".join(lines[:half]).rstrip()
            tail = "\n".join(lines[-half:]).rstrip()
            print(f"// TRUNKERT: viser første {half} og siste {half} av totalt {len(lines)} linjer")
            print(head)
            print("\n// ... [TRUNKERT] ...\n")
            print(tail)
            truncated = True
        else:
            print(text.rstrip())
    except Exception as e:
        print(f"// FEIL VED LESING: {e}")
    print("```")
    print()
    return truncated

def main():
    root = Path(".").resolve()
    files = sorted(set(iter_files(root)), key=lambda x: x.as_posix())

    # Samle meta
    git = try_git_info()
    versions = read_pkg_versions()
    env_keys = read_env_keys()

    index = []
    total_bytes = 0
    for p in files:
        try:
            sz = (root / p).stat().st_size
        except Exception:
            sz = 0
        index.append({"path": p.as_posix(), "bytes": sz})
        total_bytes += sz

    summary = {
        "project_root": str(root),
        "limits": {"max_bytes_per_file": MAX_BYTES, "max_lines_per_file": MAX_LINES},
        "counts": {"files": len(files), "sum_bytes": total_bytes},
        "git": git,
        "versions": versions,
        "env_key_files": env_keys,  # kun nøkler, ikke verdier
        "files": index,
    }

    print_header(summary)
    print_src_tree(root, files)

    # Skriv filer
    for p in files:
        print_file(root, p)

if __name__ == "__main__":
    main()


```json title=__SUMMARY__.json
{
  "project_root": "C:\\Users\\TorkelWestby\\inspiration_bank",
  "limits": {
    "max_bytes_per_file": 200000,
    "max_lines_per_file": 400
  },
  "counts": {
    "files": 30,
    "sum_bytes": 210451
  },
  "git": {
    "commit": "46bbb88d5103c7b5f758a97e95d576fdc3170b23",
    "branch": "main",
    "state": "dirty"
  },
  "versions": {
    "next": "14.2.15",
    "react": "^18.3.1",
    "react-dom": "^18.3.1",
    "node": null
  },
  "env_key_files": {
    ".env.local": [
      "AIRTABLE_BASE_ID",
      "AIRTABLE_TABLE_ID",
      "AIRTABLE_TOKEN",
      "AI_ANALYZE_ENABLED",
      "AI_MAX_REQ_PER_HOUR",
      "APP_PASSWORD",
      "APP_USERNAME",
      "NEXT_PUBLIC_ADMIN_PASSWORD",
      "NEXT_PUBLIC_ADMIN_USERNAME",
      "NEXT_PUBLIC_BASE_URL",
      "NEXT_PUBLIC_BASIC_PASSWORD",
      "NEXT_PUBLIC_BASIC_USERNAME",
      "NEXT_PUBLIC_CLOUDINARY_CLOUD_NAME",
      "NEXT_PUBLIC_CLOUDINARY_UPLOAD_PRESET",
      "OPENAI_API_KEY"
    ]
  },
  "files": [
  

In [ ]:
# from PIL import Image

# for size in [192, 512]:
#     img = Image.open("public/bama.png").convert("RGBA")  # åpne på nytt
#     img.thumbnail((size, size))

#     square = Image.new("RGBA", (size, size), (0, 0, 0, 0))
#     x = (size - img.width) // 2
#     y = (size - img.height) // 2
#     square.paste(img, (x, y), img)
#     square.save(f"public/icon-{size}.png")

# print("Lagde kvadratiske 192px og 512px ikoner uten forvrengning.")
